# Sequence to Sequence (a.k.a. seq2seq)

**학습목표**
* Encoder Decoder 구조를 이해하고 구현할 줄 안다.
* Seq2Seq에 필요한 전처리를 이해한다.
* **데이터 부족**과, **긴 문장**을 겪어본다.

![이런거](https://raw.githubusercontent.com/KerasKorea/KEKOxTutorial/master/media/28_1.png)
---------------------------------
edu.rayleigh@gmail.com
Special Thanks to : 숙번님 ( [봉수골 개발자 이선비](https://www.youtube.com/channel/UCOAyyrvi7tnCAz7RhH98QCQ) )

In [ ]:
!wget http://www.manythings.org/anki/kor-eng.zip

In [ ]:
import zipfile
kor_eng = zipfile.ZipFile('kor-eng.zip')
kor_eng.extractall()
kor_eng.close()

In [ ]:
import pandas as pd
temp = pd.read_table('kor.txt', names=['Eng', 'Kor', 'license'])
temp.shape

In [ ]:
temp.head()

In [ ]:
eng_sent = temp['Eng'].tolist()
kor_sent = temp['Kor'].tolist()

In [ ]:
print(eng_sent[1000])
print(kor_sent[1000])

# 데이터 준비
0. 단어와 구두점 사이 공백 만들기
1. sos 와 eos
1. tokenizing, idx_seq, padding

## 0. 단어와 구두점 사이 공백 만들기


In [ ]:
import unicodedata
import re
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')
def eng_preprocessor(sent):
    # 위에서 구현한 함수를 내부적으로 호출
    sent = unicode_to_ascii(sent.lower())

    # 단어와 구두점 사이에 공백을 만듭니다.
    # Ex) "he is a boy." => "he is a boy ."
    sent = re.sub(r"([?.!,'¿])", r" \1 ", sent)

    # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
    sent = re.sub(r"[^a-zA-Z!.?']+", r" ", sent)

    sent = re.sub(r"\s+", " ", sent)
    return sent

def kor_preprocessor(sent):
    # 위에서 구현한 함수를 내부적으로 호출
    sent = unicode_to_ascii(sent.lower())

    # 단어와 구두점 사이에 공백을 만듭니다.
    # Ex) "he is a boy." => "he is a boy ."
    sent = re.sub(r"([?.!,'¿])", r" \1 ", sent)

    sent = re.sub(r"\s+", " ", sent)
    return sent

In [ ]:
eng_preprocessor("I'm just a poor boy.")

In [ ]:
eng_sent = [ eng_preprocessor(sent) for sent in eng_sent ]
kor_sent = [ kor_preprocessor(sent) for sent in kor_sent ]

In [ ]:
print(eng_sent[1000])
print(kor_sent[1000])

## 1. sos 와 eos
1. sos : start of speech
2. eos : end of speech

In [ ]:
######################
### Your Code here ###
######################

## 영어 문장 전 후에 <sos>와 <eos>를 추가할 것
## 띄어쓰기 주의!

eng_sent = [f"<sos> {eng} <eos>" for eng in eng_sent]
eng_sent[1000]

## 2. Tokenizing, idx_seq, padding

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
######################
### Your Code here ###
######################

# Tokenizing    # 한국어는 lower = False
tokenizer_en = Tokenizer(filters="", lower=True)
tokenizer_en.fit_on_texts(eng_sent)
tokenizer_kr = Tokenizer(filters="", lower=False)
tokenizer_kr.fit_on_texts(kor_sent)

In [ ]:
######################
### Your Code here ###
######################

# Index Sequence
eng_seq = tokenizer_en.texts_to_sequences(eng_sent)
kor_seq = tokenizer_kr.texts_to_sequences(kor_sent)

print(eng_seq[1000])
print(kor_seq[1000])

In [ ]:
######################
### Your Code here ###
######################
## 최대 문장 길이에 맞춰지도록 할 것.
# padding
eng_pad = pad_sequences(eng_seq) # 최대 문장 길이에 패딩에 맞춰지게 됨.
kor_pad = pad_sequences(kor_seq)

print(eng_pad.shape)
print(kor_pad.shape)

In [ ]:
# tokenizer에서 0 index가 구성되어있지 않지만, 
# pad_sequence에서 pad의 의미로 0을 사용하고 있어서, 전체 사이즈를 구할 때, +1을 해준다.

eng_vocab_size = len(tokenizer_en.word_index) + 1
kor_vocab_size = len(tokenizer_kr.word_index) + 1
print("영어 단어 집합의 크기: {:d}\n한국어 단어 집합의 크기: {:d}".format(eng_vocab_size, kor_vocab_size))

# 모델링!

1. 모든 임베딩 레이어는 128개 차원으로 구성.
2. 인코더도 디코더도 GRU, 히든스테이트 512로 구성.
3. 디코더의 GRU 뒤에는 Fully Conneceted layer 사용. 노드 512개
4. 적절한 아웃풋레이어
    * 매 시점, 가장 적절한 단어가 무엇일지 분류 한다고 생각하면 됨!

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, GRU

In [ ]:
######################
### Your Code here ###
######################

# 혹시 이미 그려둔 그래프가 있다면 날려줘!
tf.keras.backend.clear_session()

# 한국어 단어 집합의 크기 : 5551, (50000, 95)
# 영어 단어 집합의 크기 : 2484, (50000, 104)
# 영어 문장은 길이가 104이지만,
# 디코더의 인풋으로 넣을때는 맨 뒤의 <eos>를 떼고 길이 103의 문장을
# 디코더의 아웃풋은 맨 앞의 <eos>를 떼고 길이 103의 문장으로 준비해야 함.

# Encoder




# Decoder




#####################
## Attention layer ##
#####################
key = enc_S_full  # 인코더의 히든스테이트를 key로 활용한다. (95, 512)
value = enc_S_full  # 인코더의 히든스테이트를 value로 활용한다. (95, 512)
query = dec_H  # 디코더의 히든스테이트를 query로 활용한다. (103, 256)

# 1. 어텐션 스코어(Attention Score)를 구한다.
# score <-- Query*Key.T


# 연산 결과: (103, 512) * (512, 95) => (103, 95)
# 영어 103개 step 각각에서  한국어 95개 step 전부의 스코어


# 2. 소프트맥스(softmax) 함수를 통해 어텐션 분포(Attention Distribution)를 구한다.
# att_dist <-- softmax(score, axis = -1)


# 연산 결과 : (103, 95) 그대로
# 위의 영어 103개 step 각각에서의 attention score들이 softmax를 통과하여 합계 1이 되었음.


# 3. 각 인코더의 어텐션 가중치와 은닉 상태를 가중합하여 어텐션 값(Attention Value)을 구한다.
# att_value <-- att_dist*value

# 연산 결과: (103, 95) * (95, 512) => (103,512)
# value의 row 들은  att_dist(softmax)값이 높은 것들이 더 가중되어 합산됨.


# # 한줄로 처리하는 마법의 코드
# att_value = tf.keras.layers.Attention()([query, key])


# 4. 어텐션 값과 디코더의 t 시점의 은닉 상태를 연결한다.(Concatenate)

# 5. 노드 512, tanh, fully connect.


# 아웃풋 레이어
dec_Y = 

model = 

# 컴파일



In [ ]:
######################
### Your Code here ###
######################
## 학습 시킬 것!




In [ ]:
import numpy as np

# 한국어 단어 집합의 크기 : 5551, (50000, 95)
# 영어 단어 집합의 크기 : 2484, (50000, 104)

def translate(kor):
    # eng => index => pad
    kor_seq = tokenizer_kr.texts_to_sequences([kor])
    kor_pad = tf.keras.preprocessing.sequence.pad_sequences(kor_seq, maxlen=95)

    eng = []
    for n in range(104-1):
        # kor => index => pad
        eng_seq = tokenizer_en.texts_to_sequences([['<sos>'] + eng])
        eng_pad = tf.keras.preprocessing.sequence.pad_sequences(eng_seq, maxlen=104-1)
        eng_next = model.predict([kor_pad, eng_pad])

        # onehot -> index -> word
        eng = [tokenizer_en.index_word[i] for i in np.argmax(eng_next[0], axis=1) if i != 0]
        # 번역된 word 선택
        eng = eng[:n+1]
        
        if eng[-1] == '<eos>':
            break

    return eng

In [ ]:
import random

# 랜덤 10개
indices = list(range(3648))
random.shuffle(indices)

for n in indices[:10]:
    print(f"한국어: {kor_sent[n]}\n영어: {eng_sent[n]}")
    print(f"번역: {' '.join(translate(kor_sent[n])[:-1])}")
    print()